In [118]:
import pandas as pd
import numpy as np

df = pd.read_excel('covid_econonomic_factors.xlsx', sheet_name = [
    'gdp_growth_rate', 'gdp_per_cap_PPP', 'personal_income_tax_rate',
    'personal_income_tax_rate', 'debt_to_gdp', 'ease_of_doing_business',
    'internet_speed', 'unemployment_rate', 'consumer_price_index', 
    'inflation_rates', 'coronavirus_cases', 'coronavirus_deaths', 
    'coronavirus_recovered', 'coronavirus_vaccination_rate', 'coronavirus_vaccination_total',
    'corruption_rank', 'population'
])

edu_factors = pd.read_excel('edu_factors.xlsx', sheet_name=[
    'gov_exp_on_edu_percent_of_gdp', 'gov_exp_on_edu_perc_per_student',
    'gov_exp_per_stud_tertiary_perc', 'pupil_teacher_ratio_primary',
    'youth_literacy_rate'
])

continent = pd.read_csv('countryContinent.csv')

In [119]:
len(df.keys())

16

In [120]:
final_df = pd.DataFrame(df['gdp_growth_rate'][['Country', 'Last']])
final_df.columns = ['Country', 'gdp_growth_rate']
for i in df.keys():
    df_tmp = df[i][['Country', 'Last']]
    df_tmp.columns = ['Country', i]
    final_df = final_df.merge(df_tmp)

In [121]:
final_df.columns

Index(['Country', 'gdp_growth_rate', 'gdp_per_cap_PPP',
       'personal_income_tax_rate', 'debt_to_gdp', 'ease_of_doing_business',
       'internet_speed', 'unemployment_rate', 'consumer_price_index',
       'inflation_rates', 'coronavirus_cases', 'coronavirus_deaths',
       'coronavirus_recovered', 'coronavirus_vaccination_rate',
       'coronavirus_vaccination_total', 'corruption_rank', 'population'],
      dtype='object')

In [122]:
final_df['covid_death_rate'] = final_df['coronavirus_deaths']/final_df['population']
final_df['covid_cases_rate'] = final_df['coronavirus_cases']/final_df['population']
final_df['covid_recovered'] = final_df['coronavirus_recovered']/final_df['population']
final_df = final_df.rename(columns={'coronavirus_vaccination_rate':'covid_vaccination_rate'})

In [123]:
final_df = continent[['country', 'continent', 'sub_region']].merge(final_df.rename(columns={'Country':'country'}), on='country')

In [124]:
final_edu = pd.DataFrame(edu_factors['gov_exp_on_edu_percent_of_gdp'][['Country', 'Most Recent Value']])
final_edu.columns = ['Country', 'gov_exp_on_edu_percent_of_gdp']
for i in edu_factors.keys():
    df_tmp = edu_factors[i][['Country', 'Most Recent Value']]
    df_tmp.columns = ['Country', i]
    final_edu = final_edu.merge(df_tmp)

final_edu = final_edu.rename(columns={'Country':'country'})

In [125]:
final_edu.country = np.where(final_edu.country == 'Bahamas, The', 'Bahamas', 
         np.where(final_edu.country == "Cote d'Ivoire", 'Ivory Coast',
         np.where(final_edu.country == "Egypt, Arab Rep.", 'Egypt',
         np.where(final_edu.country == "Gambia, The", 'Gambia',
         np.where(final_edu.country == "Hong Kong SAR, China", 'Hong Kong',
         np.where(final_edu.country == "Korea, Rep.", "South Korea",
         np.where(final_edu.country == "North Macedonia", 'Macedonia',
         np.where(final_edu.country == "Russian Federation", "Russia",
         np.where(final_edu.country == "Slovak Republic", "Slovakia", 
         final_edu.country)))))))))

In [126]:
for i in final_df.country.unique():
    if i not in final_edu.country.unique():
        print(i)

Swaziland


In [127]:
final_df = final_edu.merge(final_df, on='country')

In [128]:
final_df.to_csv('final_df.csv', index=False)